In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

xmldoc = ET.parse('tmp')

root = xmldoc.getroot()

In [2]:
Series = []
for DocSum in root.findall('DocSum'):
    data = {}
    for Item in DocSum:
        if Item.tag == 'Id':
            data['Id'] = int(Item.text)
            continue
        
        name = Item.attrib['Name']
        if name in ('Source', 'DOI', "PubDate"):
            data[name] = Item.text
    
    Series.append(pd.Series(data))

In [3]:
df = pd.concat(Series, axis=1, sort=False)

------------------------------------------

In [1]:
import re

import pandas as pd
import xml.etree.ElementTree as ET

from tqdm import tqdm_notebook as tqdm

import urllib
from urllib.parse import quote as q


In [2]:
Searches = pd.read_csv('DATA/NHMRC/searches.csv', index_col=0).drop_duplicates(subset=['CI', 'INS'])

In [3]:
import os
os.makedirs('DATA/search_data', exist_ok=True)

In [10]:
base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
db = 'pubmed' #?db=pubmed
retmax = '0'

for sid, query in tqdm(Searches.search_string.items(), total=Searches.shape[0]):
    
    if os.path.exists(f'DATA/search_data/{sid}.xml'): continue
    
    req0 = f'{base}esearch.fcgi?db={db}&term={query}&RetMax={retmax}&usehistory=y'
    
    output = urllib.request.urlopen(req0).read().decode()
    
    web, = re.findall(r'<WebEnv>(\S+)<\/WebEnv>', output)
    key, = re.findall(r'<QueryKey>(\d+)<\/QueryKey>', output)
    
    req1 = f'{base}esummary.fcgi?db={db}&query_key={key}&WebEnv={web}'
    try:
        urllib.request.urlretrieve(req1, f'DATA/search_data/{sid}.xml')
    except HTTPError: 
        pass
    

In [6]:
Searches.search_string.sample(100)

100

(2202, 4)

In [8]:
Searches.search_string

'Id'

In [ ]:
import glob

glob.glob('DATA/')

xmldoc = ET.parse('tmp')

root = xmldoc.getroot()

Series = []
for DocSum in root.findall('DocSum'):
    data = {}
    for Item in DocSum:
        if Item.tag == 'Id':
            data['Id'] = int(Item.text)
            continue
        
        name = Item.attrib['Name']
        if name in ('Source', 'DOI', "PubDate"):
            data[name] = Item.text
    
    Series.append(pd.Series(data))